# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [1]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage



# 创建ChatZhipuAI对象，指定模型为GLM-4
llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.9,
)

In [2]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
result = chain.invoke({     'product': "性能卓越的GPU"
                })
print( result )

{'product': '性能卓越的GPU', 'text': '以下是10个为制造性能卓越的GPU的有限公司取的名字，以及相应的完整公司名称：\n\n1. 霓虹之光科技有限公司\n   - 完整名称：霓虹之光高性能GPU研发科技有限公司\n\n2. 极速先锋科技有限公司\n   - 完整名称：极速先锋图形处理技术有限公司\n\n3. 虚拟视界科技有限公司\n   - 完整名称：虚拟视界高端GPU制造商有限公司\n\n4. 炫彩引擎科技有限公司\n   - 完整名称：炫彩引擎图形处理器研发有限公司\n\n5. 光影魔方科技有限公司\n   - 完整名称：光影魔方高性能GPU生产有限公司\n\n6. 图形领航科技有限公司\n   - 完整名称：图形领航先进GPU研发与制造有限公司\n\n7. 超越视觉科技有限公司\n   - 完整名称：超越视觉高端GPU技术有限责任公司\n\n8. 纵横图形科技有限公司\n   - 完整名称：纵横图形高性能GPU解决方案有限公司\n\n9. 无限创意科技有限公司\n   - 完整名称：无限创意GPU研发制造有限公司\n\n10. 星辰大海科技有限公司\n    - 完整名称：星辰大海高性能图形处理器有限公司\n\n这些名字都体现了GPU性能卓越、科技领先的特点，为公司树立了良好的形象。希望对您有所帮助！'}


In [6]:
chain.verbose =True

In [7]:
print(chain.invoke({
    'product': "性能卓越的GPU"
    })['text'])



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.
以下是10个针对制造高性能GPU的公司的好名字，以及相应的完整公司名称：

1. 宇宙星辰科技有限公司
   - 全称：宇宙星辰高性能GPU科技有限公司

2. 极速先锋电子有限公司
   - 全称：极速先锋高性能GPU电子有限公司

3. 图形领航者股份有限公司
   - 全称：图形领航者高性能GPU股份有限公司

4. 智绘科技有限公司
   - 全称：智绘高性能GPU科技有限公司

5. 炫彩视觉科技有限公司
   - 全称：炫彩视觉高性能GPU科技有限公司

6. 疾风图形技术有限公司
   - 全称：疾风图形高性能GPU技术有限公司

7. 显卡巅峰科技有限公司
   - 全称：显卡巅峰高性能GPU科技有限公司

8. 量子图形股份有限公司
   - 全称：量子图形高性能GPU股份有限公司

9. 虚拟视界科技有限公司
   - 全称：虚拟视界高性能GPU科技有限公司

10. 光影传奇科技有限公司
   - 全称：光影传奇高性能GPU科技有限公司

这些名字都体现了高性能GPU的特性，同时具有科技感和创新性，有助于塑造公司形象。希望这些建议对您有所帮助。


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [8]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。
 

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [9]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [10]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [11]:
review = overall_chain.invoke("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...
《三体人不是无法战胜的》是一部科幻题材的戏剧作品，以人类文明与外星文明三体人的碰撞为背景，探讨人类面临未知挑战时的勇气、智慧与团结。该剧以生动的情节、紧张的气氛和深刻的主题思想，展现了人类在逆境中不断追求生存与发展的信念。

简介：
在遥远的宇宙深处，一个名为三体的外星文明正密谋侵占地球。面对这股强大的敌人，地球人类似乎陷入了一场无法逃脱的浩劫。然而，在这场看似失衡的较量中，一群勇敢的科学家和战士们站了出来，他们坚信：三体人并非无法战胜。在激战与智慧的交织中，人类逐渐找到了三体文明的弱点，展开了一场惊心动魄的宇宙保卫战。本剧通过这场跨星际的较量，深刻探讨了人性、信仰、团结与爱的力量，为观众呈现出一幅波澜壮阔的科幻画卷。
《三体人不是无法战胜的》这部戏剧作品以其独特的外星文明题材和深入人心的主题思想，为观众带来了一场视觉与心灵的盛宴。在这部作品中，我们看到了人类在面临未知挑战时所展现出的勇气、智慧与团结，令人感叹不已。

该剧的剧情紧凑，气氛紧张，让人如痴如醉。在导演的巧妙安排下，观众仿佛置身于一场跨星际的较量之中，心情随着剧情的起伏而波动。无论是科学家的智慧，还是战士们的勇敢，都让人为之动容。此外，剧中对于三体文明的描绘也极具创意，令人眼前一亮。

值得一提的是，该剧深刻地探讨了人性、信仰、团结与爱的力量。在面对看似无法战胜的敌人时，人类并没有选择放弃，而是团结一心，寻找敌人的弱点，最终赢得了胜利。这种精神令人敬佩，也为现实生活中的我们带来了启示。

剧中演员的表现也相当出色，他们成功地将角色塑造得栩栩如生，让观众感同身受。特别是主演们在表现人物内心挣扎和勇敢面对困境时，演技更是达到了一个新的高度。

总的来说，《三体人不是无法战胜的》是一部富有创意、引人深思的戏剧作品。它不仅为观众带来了一场精彩绝伦的科幻盛宴，更让我们思考了人性与爱的力量。这部作品无疑是一部值得一看再看的好剧，推荐给大家。

> Finished chain.


In [ ]:
review = overall_chain.invoke("星球大战第九季")
print(review)



> Entering new SimpleSequentialChain chain...

《星球大战第九季》是一部充满惊险刺激的科幻戏剧，它延续了《星球大战》系列的传奇故事。在这一季中，我们将见证新的冒险和挑战，因为银河系再次陷入战争的深渊。反抗军和第一秩序之间的战争仍在继续，但这一次，他们将面临着更加强大的敌人。新的角色和老朋友将再次出现，与我们的英雄们一起并肩作战。随着战争的进展，我们也将见证力量和黑暗的永恒对抗，以及对自由和正义的不懈追求。在这场史诗般的战役中，我们将看到勇气、牺牲和爱的力量，同时也会思考我们对未来的选择和命运。《星球大战第九季》将带给观众无与伦比的视觉盛宴和感人心弦的故事，让我们一起来见证这一壮丽的星际冒险！


《星球大战第九季》是一部不容错过的史诗般的科幻戏剧。该剧延续了《星球大战》系列的传奇故事，将观众带入一个充满惊险和刺激的银河系。在这一季中，我们将再次与我们熟悉的英雄们一起并肩作战，同时也会见证新的挑战和冒险。

该剧不仅仅是一场视觉盛宴，更是一部让人思考的故事。通过力量和黑暗的永恒对抗，以及对自由和正义的追求，观众将被带入一个关于命运和选择的深刻思考。在这场史诗般的战役中，我们将见证勇气、牺牲和爱的力量，这些都是《星球大战》系列一直探讨的重要主题。

该剧不仅有着令人惊叹的特效和精彩的动作场面，更重要的是它成功地将这些元素与深刻的人物关系和情感表达相结合。观众将会被带入一个充满希望和感动的星际冒险，与角色们一起经历他们的成长和转变。

总的来说，《星球大战第九季》是一部充满情感和故事的出色作品，它将让观众沉浸在一个充满惊喜和挑战的星际世界。无论您是《星球大战》系列的忠实粉丝，还是对科幻题材感兴趣的观众，这部戏剧都会给您带来一场令人难忘的体验。不容错过！

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [13]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7, max_tokens=1000)
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [14]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [15]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [16]:
m_overall_chain.invoke({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


《三体人不是无法战胜的》讲述的是发生在二十一世纪新中国的故事。在这个时代，人类社会已经发展到了一个全新的境界，科技的进步让人类可以探索宇宙的奥秘。然而，就在这个时候，地球突然受到了外星种族三体人的入侵。

三体人拥有强大的科技实力和高度发达的文明，他们的出现让地球陷入了前所未有的危机之中。在三体人的攻击下，人类社会陷入混乱，无数的生命被夺走，城市被摧毁，人们被迫逃离家园。

然而，正当人类似乎无力抵抗的时候，一批勇敢的人们站了出来，他们来自不同的国家和阶层，但都有着同样的信念：三体人并不是无法战胜的。他们决定联合起来，利用自己的智慧和勇气，与三体人展开一场生死搏斗。

在这场战争中，人类付出了巨大的牺牲，但也取得了重要的胜利。最终，人类成功击退了三体人的入侵，地球重新恢复了和平与安宁。

《三体人不是无法战胜的》是一部关于人类勇气和团结的故事，它向我们展示了在面对未知和危机时，人类的坚强和不屈不挠。同时也提醒我们，科技的发展虽然能给人类带来前所未有的进步，但也需要我们谨慎使用，以免遭受意想不到的后果。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n\n《三体人不是无法战胜的》讲述的是发生在二十一世纪新中国的故事。在这个时代，人类社会已经发展到了一个全新的境界，科技的进步让人类可以探索宇宙的奥秘。然而，就在这个时候，地球突然受到了外星种族三体人的入侵。\n\n三体人拥有强大的科技实力和高度发达的文明，他们的出现让地球陷入了前所未有的危机之中。在三体人的攻击下，人类社会陷入混乱，无数的生命被夺走，城市被摧毁，人们被迫逃离家园。\n\n然而，正当人类似乎无力抵抗的时候，一批勇敢的人们站了出来，他们来自不同的国家和阶层，但都有着同样的信念：三体人并不是无法战胜的。他们决定联合起来，利用自己的智慧和勇气，与三体人展开一场生死搏斗。\n\n在这场战争中，人类付出了巨大的牺牲，但也取得了重要的胜利。最终，人类成功击退了三体人的入侵，地球重新恢复了和平与安宁。\n\n《三体人不是无法战胜的》是一部关于人类勇气和团结的故事，它向我们展示了在面对未知和危机时，人类的坚强和不屈不挠。同时也提醒我们，科技的发展虽然能给人类带来前所未有的进步，但也需要我们谨慎使用，以免遭受意想不到的后果。',
 'review': '\n\n《三体人不是无法战胜的》是一部令人惊叹的现代科幻戏剧。它不仅展现了人类社会的科技进步和探索宇宙的梦想，更重要的是通过讲述与外星种族三体人的战争，向我们传递出关于人类团结和勇气的深刻信息。\n\n该剧的故事情节紧凑、扣人心弦，将观众带入一个充满未知和危机的世界。在三体人的入侵下，人类社会陷入混乱，但也正是在这种极端的情况下，人们的团结和勇气才得以展现。从不同国家和阶层的人们联合起来，到最终成功击退入侵者，这部剧给人们带来了希望和勇气的力量。\n\n此外，该剧还提醒我们科技发展的另一面。虽然科技能为人类带来前所未有的进步，但也需要谨慎使用，以免遭受意想不到的后果。这种深刻的思考让观众在欣赏剧目的同时，也能反思自己对科技的使用态度。\n\n总的来说，《三体人不是无法战胜的》是一部令人兴奋、思考和感动的现代戏剧。它将人类的勇气和团结精神传递给观众，同时也提醒我们在面对未知和挑战时，需要保持谨慎和勇气。这部剧绝对值得一看。'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果